In [1]:
import tensorflow as tf
import keras

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, DirectoryIterator

In [2]:
basic_model = keras.models.load_model('/content/drive/MyDrive/CatClassifier/Basic CNN/ver0')
alexNet_model = keras.models.load_model('/content/drive/MyDrive/CatClassifier/AlexNet')
vgg_model = keras.models.load_model('/content/drive/MyDrive/CatClassifier/VggNet')
googLenet_model = keras.models.load_model('/content/drive/MyDrive/CatClassifier/GoogLeNet')

In [3]:
# alexnet 227 x 227
# googlenet 224 x 224
# vgg 224 x 224
# basic CNN 150 x 150

In [4]:
train_dir = '/content/drive/MyDrive/CatClassifier/train_for_test'

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
for_basic = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=1,
    shuffle=False,
    class_mode ='binary'
)

for_vgggoo = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=1,
    shuffle=False,
    class_mode ='binary'
)

for_alex = train_datagen.flow_from_directory(
    train_dir,
    target_size=(227,227),
    batch_size=1,
    shuffle=False,
    class_mode ='binary'
)

Found 8 images belonging to 2 classes.
Found 8 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [3]:
import tensorflow_datasets as tfds

In [4]:
tfds.disable_progress_bar()
vali_ds , metadata = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)

In [5]:
vali_ds = vali_ds.get('train')

In [6]:
IMG_SIZE0 = 227
def format_example0(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/255.0)
  image = tf.image.resize(image, (IMG_SIZE0, IMG_SIZE0))
  return image, label

IMG_SIZE1 = 224
def format_example1(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/255.0)
  image = tf.image.resize(image, (IMG_SIZE1, IMG_SIZE1))
  return image, label

In [7]:
val_train0 = vali_ds.map(format_example0)
val_train1 = vali_ds.map(format_example1)

In [8]:
val_train0 , val_train1 = val_train0.batch(1) , val_train1.batch(1)

In [29]:
googLenet_model.evaluate(val_train1)
# accuracy : 0.8985...

23262/23262 [==============================] - 234s 10ms/step - loss: 0.4172 - main_classifier_loss: 0.2473 - auxiliary_4a_loss: 0.3105 - auxiliary_4d_loss: 0.2557 - main_classifier_main_classifier_accuracy: 0.8985 - auxiliary_4a_auxiliary_4a_accuracy: 0.8762 - auxiliary_4d_auxiliary_4d_accuracy: 0.8970


[0.41720518469810486,
 0.24732203781604767,
 0.31053653359413147,
 0.2557348310947418,
 0.8985040187835693,
 0.8761929273605347,
 0.8969994187355042]

In [30]:
vgg_model.evaluate(val_train1)
# accuracy : 0.8933...

23262/23262 [==============================] - 171s 7ms/step - loss: 0.2532 - accuracy: 0.8933


[0.2532358169555664, 0.8933453559875488]

In [31]:
alexNet_model.evaluate(val_train0)
# accuracy : 0.9274

23262/23262 [==============================] - 85s 4ms/step - loss: 0.1809 - accuracy: 0.9274


[0.18089741468429565, 0.9274352788925171]

In [11]:
vgg = vgg_model.predict(val_train1)
goo = googLenet_model.predict(val_train1)
alex = alexNet_model.predict(val_train0)

In [12]:
pred = (vgg + goo[0] + alex) / 3
pred = tf.argmax(pred, axis = 1)

In [13]:
pred.shape

TensorShape([23262])

In [9]:
label = np.concatenate([y for x, y in val_train0], axis=0)

In [10]:
label.shape

(23262,)

In [19]:
op = pred == label
result = tf.math.count_nonzero(op)
print(result.numpy()/op.shape*100)
# accuracy : 0.9360...

[93.60330152]
